## combine 4 data files

Drop the duplicated sources, and add x,y coord (w.r.t each quadrant).

In [1]:
inst = 1
field = 'SWIRE'

datadir='../../doc/20170617_Stacking/maps/PanSTARRScats/'

df1 = pd.read_csv(datadir + 'SWIRE1_raw.csv')
df2 = pd.read_csv(datadir + 'SWIRE2_raw.csv')
df3 = pd.read_csv(datadir + 'SWIRE3_raw.csv')
df4 = pd.read_csv(datadir + 'SWIRE4_raw.csv')

In [2]:
order = [c for c in df1.columns]
df = pd.concat([df1,df2,df3,df4]).drop_duplicates(subset = 'ObjID')
df = df.drop(['ObjID'],axis=1)
order = order[1:]

In [3]:
df = df.replace(-999,np.nan)

In [4]:
print(list(df))

['ra', 'dec', 'gMeanPSFMag', 'rMeanPSFMag', 'iMeanPSFMag', 'zMeanPSFMag', 'yMeanPSFMag', 'gMeanKronMag', 'rMeanKronMag', 'iMeanKronMag', 'zMeanKronMag', 'yMeanKronMag', 'gMeanApMag', 'rMeanApMag', 'iMeanApMag', 'zMeanApMag', 'yMeanApMag', 'qualityFlag', 'objInfoFlag']


In [5]:
df['minMag'] = df[['gMeanPSFMag', 'rMeanPSFMag', 'iMeanPSFMag', 'zMeanPSFMag',\
                    'yMeanPSFMag', 'gMeanKronMag', 'rMeanKronMag', 'iMeanKronMag',\
                    'zMeanKronMag', 'yMeanKronMag', 'gMeanApMag', 'rMeanApMag',\
                    'iMeanApMag', 'zMeanApMag', 'yMeanApMag']].min(axis=1)
df['min_col'] = df[['gMeanPSFMag', 'rMeanPSFMag', 'iMeanPSFMag', 'zMeanPSFMag',\
                    'yMeanPSFMag', 'gMeanKronMag', 'rMeanKronMag', 'iMeanKronMag',\
                    'zMeanKronMag', 'yMeanKronMag', 'gMeanApMag', 'rMeanApMag',\
                    'iMeanApMag', 'zMeanApMag', 'yMeanApMag']].idxmin(axis=1)

In [6]:
df['yminMag'] = df[['yMeanPSFMag','yMeanKronMag','yMeanApMag']].min(axis=1)
df['ymin_col'] = df[['yMeanPSFMag','yMeanKronMag','yMeanApMag']].idxmin(axis=1)
order = order[0:2] + ['yminMag','ymin_col'] + order[2:]

In [7]:
df = df.sort_values('yminMag')
df = df.reset_index(drop=True)

In [8]:
df = df.replace(np.nan,-999.)

In [9]:
df = df[order]

In [10]:
df.head()

,ra,dec,yminMag,ymin_col,gMeanPSFMag,rMeanPSFMag,iMeanPSFMag,zMeanPSFMag,yMeanPSFMag,gMeanKronMag,...,iMeanKronMag,zMeanKronMag,yMeanKronMag,gMeanApMag,rMeanApMag,iMeanApMag,zMeanApMag,yMeanApMag,qualityFlag,objInfoFlag
0,241.093416,55.335304,6.49163,yMeanPSFMag,-999.00000,-999.00000,-999.0,-999.00000,6.49163,-999.0000,...,-999.0,-999.000,12.9557,-999.0000,-999.0000,-999.0,-999.0000,17.439899,32,268513280
1,242.573392,54.657916,6.90106,yMeanPSFMag,8.84053,8.17406,-999.0,-999.00000,6.90106,12.3405,...,-999.0,-999.000,10.4736,15.6011,15.3494,-999.0,-999.0000,13.094000,160,1343750144
2,242.573199,54.658166,7.00679,yMeanPSFMag,10.15500,-999.00000,-999.0,-999.00000,7.00679,12.5871,...,-999.0,-999.000,-999.0000,13.5589,-999.0000,-999.0,-999.0000,-999.000000,32,268959744
3,242.537149,54.359664,7.08880,yMeanPSFMag,-999.00000,-999.00000,-999.0,-999.00000,7.08880,-999.0000,...,-999.0,-999.000,13.6783,-999.0000,-999.0000,-999.0,-999.0000,18.206600,32,268513280
4,244.272341,55.269016,7.53716,yMeanPSFMag,10.75600,9.50143,-999.0,8.16382,7.53716,12.8689,...,-999.0,11.075,10.1184,13.6847,13.8894,-999.0,14.5503,13.785300,32,268959744


## Add x,y coord

In [11]:
import astropy.io.fits as fits
import astropy.wcs as wcs
from astropy import units as u
from astropy.coordinates import SkyCoord

In [12]:
hdrdir = '/Users/ytcheng/ciber/doc/20170617_Stacking/maps/astroutputs/inst1/'
order = order[:2] + ['x','y'] + order[2:]

for quad in ['A','B','C','D']:
    hdulist = fits.open(hdrdir + 'SWIRE_'+quad+'_astr.fits')
    wcs_hdr=wcs.WCS(hdulist[('primary',1)].header, hdulist)
    hdulist.close()
    src_coord = SkyCoord(ra=df['ra']*u.degree, dec=df['dec']*u.degree, frame='icrs')
    print('get src_coord')
    
    x_arr, y_arr = wcs_hdr.all_world2pix(df['ra'],df['dec'],0)
    df['x'] = x_arr
    df['y'] = y_arr
    df = df[order]
    
    df.to_csv(datadir + 'SWIRE_'+quad+'.csv',index=False)
    
    print('write new df {} cols in quad {}'.format(len(df),quad))

get src_coord
write new df 1229661 cols in quad A
get src_coord
write new df 1229661 cols in quad B
get src_coord
write new df 1229661 cols in quad C
get src_coord
write new df 1229661 cols in quad D
